**Querying PDF With Astra and LangChain**

Install the required dependencies

In [1]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 860.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

Importing the packages

In [2]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# dataset with Hugging face
from datasets import load_dataset

#DB connection , Astra DB integration in langChain
import cassio


In [45]:
!pip install --upgrade astrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.3 MB/s eta 0:00:00


In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00


In [4]:
#to read pdf
from PyPDF2 import PdfReader

# Setup

In [31]:
#Astra DB connection and OpenAI API Key details
Astra_DB_Token = "AstraCS:gHzEQWWJlWWJScTPobbiyCGX:9882a1076e5f5f160750cbbf7b7233ffba46fc7078ef5ee1866732bc9d1b21e4"

Astra_DB_ID = "c6b29560-43f5-43b0-91e1-76c42242ca1e"
OPENAI_API_Key = "#use your own OpenAI API key"


Diagram

In [9]:
#path to pdf file

pdfreader = PdfReader('/content/ml_notes.pdf')

In [10]:
#raw text will contain the enteir chunk of text
from typing_extensions import Concatenate
raw_text=''
for i,page in enumerate(pdfreader.pages):
   content = page.extract_text()
   if content:
     raw_text += content

In [11]:
raw_text

'INTRODUCTION TO MACHINE LEARNINGIntroduction to Machine Learning\nAlex Smola and S.V.N. Vishwanathan\nYahoo! Labs\nSanta Clara\n{and{\nDepartments of Statistics and Computer Science\nPurdue University\n{and{\nCollege of Engineering and Computer Science\nAustralian National University\npublished by the press syndicate of the university of cambridge\nThe Pitt Building, Trumpington Street, Cambridge, United Kingdom\ncambridge university press\nThe Edinburgh Building, Cambridge CB2 2RU, UK\n40 West 20th Street, New York, NY 10011{4211, USA\n477 Williamstown Road, Port Melbourne, VIC 3207, Australia\nRuiz de Alarc\x13 on 13, 28014 Madrid, Spain\nDock House, The Waterfront, Cape Town 8001, South Africa\nhttp://www.cambridge.org\nc\rCambridge University Press 2008\nThis book is in copyright. Subject to statutory exception\nand to the provisions of relevant collective licensing agreements,\nno reproduction of any part may take place without\nthe written permission of Cambridge University Pres

Connection to Database

In [12]:
# cassio integrates Apache Cassandra with ML/LLM/genAI workloads
cassio.init(token=Astra_DB_Token,database_id=Astra_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(134350005757664) c6b29560-43f5-43b0-91e1-76c42242ca1e-us-east1.db.astra.datastax.com:29042:d964cd65-e324-4903-9b1e-d4ba799e7610> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Creating the Langchain embedding & LLM objects

In [32]:
 llm = OpenAI(openai_api_key=OPENAI_API_Key)
 embedding =OpenAIEmbeddings(openai_api_key=OPENAI_API_Key )

LangChain vector store

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [21]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [22]:
texts[:40]

['INTRODUCTION TO MACHINE LEARNINGIntroduction to Machine Learning\nAlex Smola and S.V.N. Vishwanathan\nYahoo! Labs\nSanta Clara\n{and{\nDepartments of Statistics and Computer Science\nPurdue University\n{and{\nCollege of Engineering and Computer Science\nAustralian National University\npublished by the press syndicate of the university of cambridge\nThe Pitt Building, Trumpington Street, Cambridge, United Kingdom\ncambridge university press\nThe Edinburgh Building, Cambridge CB2 2RU, UK\n40 West 20th Street, New York, NY 10011{4211, USA\n477 Williamstown Road, Port Melbourne, VIC 3207, Australia\nRuiz de Alarc\x13 on 13, 28014 Madrid, Spain\nDock House, The Waterfront, Cape Town 8001, South Africa\nhttp://www.cambridge.org\nc\rCambridge University Press 2008',
 'Ruiz de Alarc\x13 on 13, 28014 Madrid, Spain\nDock House, The Waterfront, Cape Town 8001, South Africa\nhttp://www.cambridge.org\nc\rCambridge University Press 2008\nThis book is in copyright. Subject to statutory exception\na

Load the dataset into the vector store

In [ ]:
astra_vector_store.add_texts(texts[:40])
print("Text %i inserted " % len(texts[:50]))
astra_vector_index =VectorStoreIndexWrapper(vectorstore= astra_vector_store)

In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))